In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score

from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV  #Perforing grid search

import re 

In [2]:
train_limpio = pd.read_csv("train_limpio.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,"['our', 'deeds', 'are', 'the', 'reason', 'of',...",1,0,0,1,70,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...",1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

,id,target
0,0,0
1,2,0


In [4]:
test_limpio = pd.read_csv("test_limpio.csv",encoding = "ISO-8859-1")
test_limpio_sin_texto = test_limpio.drop(['text','keyword','location'], axis=1)
test_limpio_sin_texto.head()

,id,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,of,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,0,0,0,1,40,0,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,2,0,0,1,70,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,3,0,0,1,100,1,1,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,9,0,0,1,40,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,11,0,0,1,50,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0


In [5]:
df_x=train_limpio.drop(['text', 'target'], axis=1)
df_x.head()

,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,the,a,to,in,of,and,...,plans to,police post,provoke outrage,read by,rescuers are,letras_seguidas,insultos_por_tweet,expresiones_por_tweet,kill,died
0,0,0,1,70,1,0,0,0,1,0,...,0,0,0,0,0,2,0,0,0,0
1,0,0,1,40,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,130,0,0,1,2,0,0,...,0,0,0,0,0,2,0,0,0,0
3,0,0,1,60,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,1,90,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0


In [6]:
df_y=train_limpio.target
df_y

0       1
1       1
2       1
3       1
4       1
       ..
7129    1
7130    1
7131    1
7132    1
7133    1
Name: target, Length: 7134, dtype: int64

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [8]:
gbl = GradientBoostingClassifier()
gbl.fit(x_train, y_train)

GradientBoostingClassifier()

In [9]:
pred = gbl.predict(x_test)
pred

array([1, 0, 0, ..., 0, 0, 0])

In [10]:
f1_score(y_test.values, pred, average='macro')

0.7026261574873887

In [11]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [12]:
train_sin_texto = train_limpio.drop('text',axis=1)

In [13]:
#parameters = {
    #'n_estimators':[110,120,130,140,150,160,170],
#    'n_estimators':[160],  #el mejor learning rate es 0.2, entonces me 
                #conviene subirlo mientras dejo los otros parametros iguales
#    'learning_rate': [0.15, 0.2, 0.3]
    #"min_samples_split": np.linspace(0.1, 0.5, 6)
    #"min_samples_leaf": np.linspace(0.1, 0.5, 6),
 #   }
#clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)
#clf.fit(x_train, y_train)
#print(clf.score(x_train, y_train))
#print(clf.best_params_)

In [14]:
#Uso los parametros del modelo anterior
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    gbl = GradientBoostingClassifier(learning_rate= 0.3, n_estimators= 160)
    gbl.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = gbl.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = gbl.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

KeyboardInterrupt: 

In [ ]:
#Uso los parametros del modelo anterior
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    gbl = GradientBoostingClassifier(learning_rate= 0.4, n_estimators= 160)
    gbl.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = gbl.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = gbl.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

In [ ]:
#Uso los parametros del modelo anterior
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    gbl = GradientBoostingClassifier(learning_rate= 0.5, n_estimators= 160)
    gbl.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = gbl.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = gbl.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

In [ ]:
#Uso los parametros del modelo anterior
for train_index,test_index in kf.split(train_sin_texto):
    x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
    y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
    gbl = GradientBoostingClassifier(learning_rate= 0.6, n_estimators= 160)
    gbl.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = gbl.predict(x_train)
    print('Error de entrenamiento: ',f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = gbl.predict(x_test)
    print('Error de test: ',f1_score(y_test.values, pred, average='macro'))

In [ ]:
#Ya acá empiezo a overfittear
# El óptimo del learning rate esta entre 0,4 y 0.6

In [ ]:
for i in np.arange(0.4,0.61,.05):
    print(i)

In [ ]:
np.arange(0.4,0.6,0.5)

In [ ]:
best_score = 0
best_value = 0
current_score=[]
for i in np.arange(0.45,0.51,.01):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        gbl = GradientBoostingClassifier(learning_rate= i, n_estimators= 160)
        gbl.fit(x_train, y_train)
       
        #error de test:        
        pred = gbl.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        print('Error de test: ',score)
        
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_value = i
    print('')
    

print('Termine todo y la mejor iteracion fue con learning rate de:', best_value)    

In [ ]:
best_score = 0
best_value = 0
current_score=[]
for i in np.arange(150,171,5):
    print('Estoy en la iteracion',i)
    current_score = []
    for train_index,test_index in kf.split(train_sin_texto):
        x_train,x_test = train_sin_texto.drop(['target'], axis=1).loc[train_index], train_sin_texto.drop([ 'target'], axis=1).loc[test_index]
        y_train,y_test = train_sin_texto.target[train_index], train_sin_texto.target[test_index]
    
        gbl = GradientBoostingClassifier(learning_rate= 0.48, n_estimators= i)
        gbl.fit(x_train, y_train)
       
        #error de test:        
        pred = gbl.predict(x_test)
        score = f1_score(y_test.values, pred, average='macro')
        print('Error de test: ',score)
        
        current_score.append(score)
        
    print('El error total es:', sum(current_score)/len(current_score))
    if(sum(current_score)/len(current_score)>best_score):
        best_score = sum(current_score)/len(current_score)
        best_value = i
    print('')
    

print('Termine todo y la mejor iteracion fue con learning rate de:', best_value)    

In [34]:
# Entonces, mi modelo optimizado es:
train_x=train_limpio.drop(['text', 'target'], axis=1)

train_y=train_limpio.target



gbl = GradientBoostingClassifier(learning_rate= 0.48,n_estimators= 160,min_samples_split=12,min_samples_leaf=1)
gbl.fit(x_train, y_train)

    
#error de entrenamiento, es el unico que puedo calcular:
pred = gbl.predict(train_x)
print('Error de entrenamiento: ',f1_score(train_y, pred, average='macro'))


Error de entrenamiento:  0.8396278112858315


In [26]:
pred = gbl.predict(test_limpio_sin_texto.drop(['id'],axis=1))

In [27]:
sample.target=pred

In [28]:
sample.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [29]:
sample.to_csv('prediccion_gradient_boosting.csv', index=False)

# Revisar, está bien qu no sea xgboost, pero no tiene sentido que random forest me de mejor que este

In [ ]:
#https://www.kaggle.com/hatone/gradientboostingclassifier-with-gridsearchcv
#https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/